This notebook is inpired by [Detailed EDA by Erik Bruin](https://www.kaggle.com/erikbruin/house-prices-lasso-xgboost-and-a-detailed-eda/report#loading-and-exploring-data)


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm, skew
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read training and testing data sets
train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')
all_data = pd.concat([train, test], axis='index')

In [ ]:
# Shape of data
print("Training Data Shape: ",train.shape)
print("Testing Data Shape: ", test.shape)

In [ ]:
# data info
train.info()

In [ ]:
# Split data into Input variables and Target Variable
# remove the Id column from training data (not useful)
y_train = train['SalePrice']
X_train = train.drop(['SalePrice', 'Id'], axis='columns')

# remove Id column from testing data but keep it for submission
X_test = test.drop(['Id'], axis='columns')
X_test_ids = test['Id']

In [ ]:
# Explore some stats about our target variable SalePrice
y_train.describe()

In [ ]:
# Explore the distribution of the target variable
sns.histplot(data= y_train)

In [ ]:
# it is apparently not normally distributed, let's see how far it is from normal distribution
# using Q–Q (quantile-quantile) plot
fig = plt.figure()
res = stats.probplot(x = y_train, plot= plt, dist='norm')
plt.show()

In [ ]:
# divide columns into categorical and numeric
cat_cols = X_train.select_dtypes('object').columns.to_list()
num_cols = X_train.select_dtypes('number').columns.to_list()

In [ ]:
# Check Numeric Variables First
print(f'There are {len(num_cols)} numeric variables') # excluding the Id column

# Let's see how they correlate with the target variable
num_corr_target = X_train[num_cols].corrwith(y_train, axis='index').sort_values(ascending=False)

# Let's see how many highly correlate (has correlation coefficient >= |0.5|)
print(f'Numeric features with high correlation (>= |0.5|) to target variable:')
high_num_corr_target = num_corr_target[num_corr_target >= abs(0.5)]
print(high_num_corr_target)

high_num_corr_target_matrix = train[high_num_corr_target.index.to_list() + ['SalePrice']].corr()
high_num_corr_target_matrix

In [ ]:
# Visualize the correlation matrix using heatmap
fig, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(high_num_corr_target_matrix, annot= True)
plt.show()

# Observations:
# OverallQual, GrLivArea, GarageCars are the top 3 correlated features
# There is multicollinearity in the data. For example GarageCars and GarageArea are highly correlated (0.88)
# and both of them are highly correlated with the SalePrice.
# If you think about it, the space of garage and number of cars that can fit are closely related to each other.

In [ ]:
# Having a closer look on the most correlated feature (OverallQual)
fig, ax = plt.subplots(figsize=(11, 9))
sns.boxplot(x="OverallQual", y="SalePrice", data=train)
plt.show()

# The positive correlation is obvious with a slightly upward curve.
# Which indicates that as the overall quality of the house increases the price increases
# Except for 2 candidate outliers (in my opinion), there are no extreme cases.
# 1st candidate: expensive house in level 4
# 2nd candidate: expensive house in level 8

In [ ]:
# Having a closer look on the second most correlated feature (GrLivArea)
fig, ax = plt.subplots(figsize=(11, 9))
# sns.scatterplot(x="GrLivArea", y="SalePrice", data=train)
sns.regplot(x="GrLivArea", y="SalePrice", data=train, ci=None, line_kws={"color":"black"})
plt.show()

# The positive linear relation is quite obvious here as well
# Which indicates that as the (above ground) living area increases the price increases as well
# There are 2 outliers (Id:1289 & 523) with huge living areas but very low prices (<200,000)
X_train['GrLivArea'].sort_values(ascending=False).head(2)

In [ ]:
# Check completeness of data
col_has_null = all_data.isna().sum()
col_w_null = col_has_null[col_has_null > 0].index.to_list()
col_has_null[col_has_null > 0].sort_values(ascending = False)

In [ ]:
# Primary strategies for filling missing data (will see as we go if we need other strategies)
# For numerical features: assign 0
# For categorical features: assign None

# I make a copy so that I can compare before and after
all_data_original = all_data.copy()

# I will make a list of ordinal and categorical features that will need encoding
ordinal_features = []
nominal_features = []

In [ ]:
# From the description of features, we find that there are multiple columns that use the same quality measures
# So we will encode them to an ordinal variable, so we can apply it to the different columns
quality_ordinal_encoder = OrdinalEncoder(categories=
                                         [['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                         ['Reg' 'IR1' 'IR2' 'IR3']])

In [ ]:
# Pool Data : Quality and Area
# From the data description we know that Quality is categorical where NA means no pool
# So we can assign value None instead
all_data['PoolQC'] = all_data['PoolQC'].fillna('None')
print("PoolQC after filling missing values")
print(all_data[['PoolQC']].value_counts())

In [ ]:
# Now let's check if there are pools that have no quality
all_data[(all_data['PoolQC'] == 'None') & (all_data['PoolArea'] > 0 )][['Id', 'PoolArea', 'PoolQC']]
# There are 3 houses !

In [ ]:
# A nice idea by ERIK BRUIN is to infer the quality of pool from the overall quality of the whole house
all_data[(all_data['PoolQC'] == 'None') & (all_data['PoolArea'] > 0 )][['Id', 'PoolArea', 'PoolQC', 'OverallQual']]

In [ ]:
# Assign the quality of pool to the mean of overall quality (overall quality scale is twice of the pool quality scale)
all_data.loc[all_data['Id'] == 2421, 'PoolQC'] = 'Fa'
all_data.loc[all_data['Id'] == 2504, 'PoolQC'] = 'TA'
all_data.loc[all_data['Id'] == 2600, 'PoolQC'] = 'Fa'

all_data[all_data['Id'].isin([2421,2504,2600])][['Id', 'PoolArea', 'PoolQC', 'OverallQual']]

# PoolQC will be ordinal encoded
ordinal_features.append('PoolQc')

In [ ]:
# Misc Features
# From the data description we know that Misc is categorical where NA means no extra features
# So we can assign value None instead
all_data['MiscFeature'] = all_data['MiscFeature'].fillna('None')
print("MiscFeature after filling missing values")
all_data[['MiscFeature']].value_counts()

# MiscFeature will be one hot encoded
nominal_features.append('MiscFeature')

In [ ]:
# Alley
# From the data description we know that Alley is categorical where NA means no alley access
# So we cann assign value None instead
all_data['Alley'] = all_data['Alley'].fillna('None')
print('Alley after filling missing values')
all_data['Alley'].value_counts()

# Alley will be one hot encoded
nominal_features.append('Alley')

In [ ]:
# Fence
# From the data description we know that Fence is categorical where NA means no Fence
# So we cann assign value None instead
all_data['Fence'] = all_data['Fence'].fillna('None')
print('Fence after filling missing values')
all_data['Fence'].value_counts()

# Fence will be one hot encoded
nominal_features.append('Fence')

In [ ]:
# FirePlace Quality
# From the data description we know that FirePlace Quality is categorical where NA means no Fire Place
# So we cann assign value None instead
all_data['FireplaceQu'] = all_data['FireplaceQu'].fillna('None')
print('FireplaceQu after filling missing values')
all_data['FireplaceQu'].value_counts()

In [ ]:
# Now let's check if there are Fireplaces that have no quality
all_data[(all_data['FireplaceQu'] == 'None') & (all_data['Fireplaces'] > 0)][['Id', 'Fireplaces', 'FireplaceQu']]
# There are none ! Everything looks good

# FireplaceQu will be ordinal encoded
ordinal_features.append('FireplaceQu')

In [ ]:
# LotFrontage
# We know from the description that LotFrontage is numerical where NA is a missing value
# Another nice idea by ERIK BRUIN is to infer the LotFrontage from the values of the neighborhood
# But first let's check that all the neighborhoods have at least one value
print (all_data['Neighborhood'].nunique() == all_data[all_data['LotFrontage'].notna()]['Neighborhood'].nunique())

In [ ]:
# Yes looks great! Now, let's assign the median value of LotFrontage from the same neighborhood
all_data['LotFrontage'] = all_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

In [ ]:
# Let's also have a look on other Lot features (LotShape, LotConfig)

# LotShape
# from the descriptions it looks like an ordinal feature
print(all_data['LotShape'].unique()) #  Add values to ordinal encoder
ordinal_features.append('LotShape')

#LotConfig
print(all_data['LotConfig'].unique())
# Doesn't seem like an ordinal feature, so will consider as nominal
nominal_features.append('LotConfig')

In [ ]:
# Garage
# There are multple Garage variables with missing values
# GarageCond       159
# GarageYrBlt      159
# GarageFinish     159
# GarageQual       159
# GarageType       157
# In addition to 2 variables with 1 missing data
# GarageCars       1
# GarageArea       1

# First let's check if the nulls happen for the same records
all_data[(all_data['GarageCond'].isna()) & (all_data['GarageYrBlt'].isna()) & (all_data['GarageFinish'].isna()) & (all_data['GarageQual'].isna()) & (all_data['GarageType'].isna())].shape
# So 157 out of 159 are common

In [ ]:
# Now let's see the 2 cases where the difference happens
all_data[(all_data['GarageType'].notnull()) & (all_data['GarageCond'].isna())][['Id','GarageCond','GarageYrBlt','GarageFinish','GarageQual','GarageType','GarageCars','GarageArea', 'YearBuilt']]

# House 2127 looks like it has a garage (figure out how to fill the values)
# House 2577 looks like it doesn't have a garage (adjust values to reflect no garage)

In [ ]:
# For house 2127, I will consider the GarageYrBlt equal to YearBuilt (1910)
# I will check the most common properties of Garages for houses built at that year
all_data[all_data['YearBuilt'] == 1910].groupby(['GarageCond','GarageYrBlt','GarageFinish','GarageQual']).size().sort_values(ascending=False)

'GarageCond',
'GarageYrBlt',
'GarageFinish',
'GarageQual',
'GarageType',
'GarageCars'
'GarageArea'

In [ ]:
# GarageCond
# from the descriptions it is an ordinal feature